In [1]:
%matplotlib qt
import math
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

# Generate original image

In [2]:
N0, N = 50, 32
I0 = 0.1 * np.ones((N0, N0))

for r in range(10, 25):
    for c in range(14, 29):
        I0[r][c] = 1.2

for r in range(15, 34):
    for c in range(17, 23):
        I0[r][c] = 1.6
    for c in range(32, 37):
        I0[r][c] = 1.2
        
for r in range(34, 40):
    for c in range(8, 42):
        I0[r][c] = 2

# Show original image
fig1 = plt.figure(figsize=(9,6), dpi=100)
plt.suptitle('Figure 2.5')
ax1 = fig1.add_subplot(121)
plt.imshow(I0, cmap='gray_r', vmin=0, vmax=2)
plt.colorbar()
plt.show()

# Define forward model
Generate forward model $Ax = y$, where x, y $\in \mathbb{R}^{N^2}$ and A $\in \mathbb{R}^{N^2 \times N^2}$.
$$
\phi(x) = e^{-\alpha |x|}
$$
$$
g(p_{jk}) \approx \sum_{l,m = 1}^{N} \Delta^2 \phi(p_{jk} - p_{lm}) f(p_{lm})
$$

In [3]:
# Function to calculate phi for points p_jk and p_lm
def phi(p_jk, p_lm, alpha):
    dist = math.sqrt((p_jk[0] - p_lm[0]) ** 2 + (p_jk[1] - p_lm[1]) ** 2)
    return math.exp(-alpha * dist)

In [4]:
# Make a grid for the image
Delta0, alpha = 1 / N0, 20
u0 = v0 = np.arange(0, 1, Delta0) + Delta0 / 2
uu0, vv0 = np.meshgrid(u0, v0)
U0 = uu0.reshape((-1,1))
V0 = vv0.reshape((-1,1))

# Evaluate the convolution matrix A
N02 = len(U0)
A0 = np.zeros((N02, N02))
for jk, p_jk in enumerate(zip(U0, V0)):
    for lm, p_lm in enumerate(zip(U0, V0)):
        A0[jk][lm] = phi(p_jk, p_lm, alpha) * Delta0 ** 2

In [5]:
# Get blurred image after convolution with phi
x0 = I0.reshape((-1, 1))
y0 = A0 @ x0

# Add Gaussian noise to the blurred image
sigma = 0.01 * np.max(y0)
e = np.random.normal(scale=sigma, size=N02).reshape(-1, 1)
y0_noisy = y0 + e
I0_noisy = y0_noisy.reshape((N0, N0))

# Downsample from N0 to N
I = resize(I0_noisy, (N, N))
y = I.reshape((-1, 1))

# Show noisy blurred image
ax2 = fig1.add_subplot(122)
plt.imshow(I, cmap='gray_r')
plt.colorbar()
plt.show()

# Solving the inverse solution using Tikhonov Regularization

In [6]:
# Make a grid for the image
Delta, alpha = 1 / N, 20
u = v = np.arange(0, 1, Delta) + Delta / 2
uu, vv = np.meshgrid(u, v)
U = uu.reshape((-1,1))
V = vv.reshape((-1,1))

# Evaluate the convolution matrix A
N2 = len(U)
A = np.zeros((N2, N2))
for jk, p_jk in enumerate(zip(U, V)):
    for lm, p_lm in enumerate(zip(U, V)):
        A[jk][lm] = phi(p_jk, p_lm, alpha) * Delta ** 2

In [7]:
# Noise level
eps = N * sigma
print('ε = %f' % eps)

# Test values of regularization parameter
Omega = np.geomspace(1e-7, 1e-5, num=9)

# Find solutions for all test values of regularization parameter
err = np.zeros((len(Omega), 1))
fig2 = plt.figure(figsize=(9,9))
plt.suptitle('Figure 2.7')
for i, omega in enumerate(Omega):
    # Calculate inverse
    U, L, VT = np.linalg.svd(A, full_matrices=0)
    A_inv = VT.T @ np.diag([l / (l ** 2 + omega) for l in L]) @ U.T

    # Recovered image
    x_omega = A_inv @ y
    I_omega = x_omega.reshape((N, N))
    err[i] = np.linalg.norm(A @ x_omega - y)
    
    # Show image
    ax = fig2.add_subplot(3, 3, i+1)
    plt.title('$\delta$ = %f' % omega)
    plt.imshow(I_omega, cmap='gray_r')
    plt.colorbar()
plt.show()

plt.figure(figsize=(9,6))
plt.title('Figure 2.6')
plt.plot(Omega, err, '-s')
plt.axhline(y=eps, ls='-', color='k')
plt.xlabel('$\delta$', fontsize=14)
plt.ylabel('$|A x_\delta - y|$', fontsize=14)
plt.show()

ε = 0.007165
